In [48]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

%matplotlib inline

In [226]:
ds1 = pd.read_csv("Data/movies_metadata.csv")
ds1.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [333]:
vote = ds1[['id', 'vote_average']]
indexNames = vote[vote['id'].str.isdigit() == False].index
vote.drop(indexNames, inplace=True)
vote['id'] = vote['id'].astype(int)

/Users/waynsday/miniconda3/envs/UPDEDA/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/waynsday/miniconda3/envs/UPDEDA/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### Genres

In [576]:
genre = ds1[["genres", "id", "vote_average"]]
genre["genredict"] = genre.apply(lambda row : eval(row['genres']), axis = 1)
genre.drop('genres', inplace = True, axis =1)
genre

/Users/waynsday/miniconda3/envs/UPDEDA/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,vote_average,genredict
0,862,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,5.7,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...
45461,439050,4.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,111109,9.0,"[{'id': 18, 'name': 'Drama'}]"
45463,67758,3.8,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,0.0,[]


In [572]:
newlist = [item for item in genre.genredict[0] if 'name' in item]
genres += [item.get('name') for item in newlist if item.get('name') not in genres]
genres

['Adventure', 'Fantasy', 'Family', 'Animation', 'Comedy']

In [645]:
genres = []
def getGenreRating(row, genre):
    a = next((item for item in row if item['name']== genre), None)
    if a is None:
        return False
    else:
        return True
    
def getGenres(row):
    global genres
    newlist = [item for item in row if 'name' in item]
    genres += [item.get('name') for item in newlist if item.get('name') not in genres]


In [646]:
genre.genredict.apply(lambda x: getGenres(x))
genres

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'History',
 'Science Fiction',
 'Mystery',
 'War',
 'Foreign',
 'Music',
 'Documentary',
 'Western',
 'TV Movie',
 'Carousel Productions',
 'Vision View Entertainment',
 'Telescene Film Group Productions',
 'Aniplex',
 'GoHands',
 'BROSTA TV',
 'Mardock Scramble Production Committee',
 'Sentai Filmworks',
 'Odyssey Media',
 'Pulser Productions',
 'Rogue State',
 'The Cartel']

In [648]:
for i in genres:
    genre[i] = genre.genredict.apply(lambda x: getGenreRating(x,i))
genre

,id,vote_average,genredict,x,Animation,Comedy,Family,Adventure,Fantasy,Romance,...,Telescene Film Group Productions,Aniplex,GoHands,BROSTA TV,Mardock Scramble Production Committee,Sentai Filmworks,Odyssey Media,Pulser Productions,Rogue State,The Cartel
0,862,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Animation,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,8844,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",None,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2,15602,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",None,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,31357,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,11862,5.7,"[{'id': 35, 'name': 'Comedy'}]",None,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,439050,4.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",None,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
45462,111109,9.0,"[{'id': 18, 'name': 'Drama'}]",None,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
45463,67758,3.8,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",None,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
45464,227506,0.0,[],None,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [650]:
genrerank = dict.fromkeys(genres) 
for i in genres:
    d = genre[[i, 'vote_average']]
    e = d[d[i] == True]
    genrerank[i] = e['vote_average'].mean()
genredf = pd.DataFrame.from_dict(genrerank, orient='index', columns ={'vote_average'})
genredf.dropna(inplace=True)
genredf['Final Rank'] = genredf['vote_average'].rank(ascending = False)
genredf.sort_values(by='Final Rank', ascending = True, inplace=True)
genredf.to_csv('Data/Genre Ranking.csv')
genredf.to_csv('Genre Ranking.csv')
genredf

,vote_average,Final Rank
Animation,6.275181,1.0
History,6.154220,2.0
War,6.041119,3.0
Drama,5.905221,4.0
Music,5.879599,5.0
Crime,5.878407,6.0
Documentary,5.823093,7.0
Mystery,5.820470,8.0
Romance,5.802658,9.0
Fantasy,5.787722,10.0


### Language

In [670]:
lang = ds1[['original_language','vote_average']]
indexNames = lang[lang['original_language'].str.isdigit() == True].index
lang = lang.drop(indexNames)
a = lang.dropna().groupby('original_language').mean()
b = lang.dropna().groupby('original_language').count()
c = lang.dropna().groupby('original_language').sum()
poplang = pd.merge(a,b, on=['original_language'])
poplang = pd.merge(poplang,c, on=['original_language'])
poplang.rename(columns = {'vote_average_x': 'vote_averages', 'vote_average_y': 'no. of entries','vote_average': 'sum of scores'}, inplace = True)
poplang

,vote_averages,no. of entries,sum of scores
original_language,,,
ab,5.000000,10,50.0
af,6.200000,2,12.4
am,7.450000,2,14.9
ar,6.817949,39,265.9
ay,6.000000,1,6.0
...,...,...,...
vi,5.880000,10,58.8
wo,6.520000,5,32.6
xx,5.854545,33,193.2


In [672]:
poplang['Rank (Vote average)'] = poplang['vote_averages'].rank(ascending = False)
poplang['Rank (Sum)'] = poplang['sum of scores'].rank(ascending = False)
poplang['Average Rank'] = (poplang['Rank (Vote average)'] + poplang['Rank (Sum)'])/2
poplang.sort_values(by=['Average Rank'], ascending = True, inplace= True)
poplang['Final Ranking'] = poplang['Average Rank'].rank()

In [673]:
poplang.to_csv('Data/Language Ranking.csv')
poplang.to_csv('Language Ranking.csv')
poplang

,vote_averages,no. of entries,sum of scores,Rank (Vote average),Rank (Sum),Average Rank,Final Ranking
original_language,,,,,,,
ko,6.570045,444,2917.1,20.0,9.0,14.5,1.0
ja,6.274203,1349,8463.9,35.0,3.0,19.0,2.5
bn,7.137931,29,207.0,6.0,32.0,19.0,2.5
ta,6.692308,78,522.0,15.0,24.0,19.5,4.0
tr,6.537333,150,980.6,22.0,18.0,20.0,5.5
...,...,...,...,...,...,...,...
si,3.400000,1,3.4,84.0,85.0,84.5,85.0
mt,0.000000,1,0.0,87.5,87.5,87.5,87.5
uz,0.000000,1,0.0,87.5,87.5,87.5,87.5


### Director

In [252]:
ds2 = pd.read_csv("Data/credits.csv")
ds2.shape

(45476, 3)

In [283]:
def getDirek(row):
    a = next((item for item in row if item['job'] == "Director"), None)
    if a is None:
        return None
    else:
        return a['name']

In [360]:
director = ds2[["crew", "id"]]
director["crewdict"] = director.apply(lambda row : eval(row['crew']), axis = 1)
director.drop('crew', inplace = True, axis =1)
director['direk'] = director.crewdict.apply(lambda row : getDirek(row))
director.drop('crewdict', inplace = True, axis = 1)

In [373]:
merged = pd.merge(director, vote, on=['id'] )
merged.drop('id', inplace = True, axis = 1)

In [437]:
a = merged.groupby('direk').mean()
b = merged.groupby('direk').count()
c = merged.groupby('direk').sum()
bestdirek = pd.merge(a,b, on=['direk'])
bestdirek = pd.merge(bestdirek,c, on=['direk'])
bestdirek.rename(columns = {'vote_average_x': 'vote_averages', 'vote_average_y': 'no. of entries','vote_average': 'sum of scores'}, inplace = True)
bestdirek

,vote_averages,no. of entries,sum of scores
direk,,,
Dale Trevillion\t,2.0,2,4.0
Davide Manuli,6.9,1,6.9
E.W. Swackhamer,5.9,1,5.9
Vitaliy Vorobyov,5.5,1,5.5
Yeon Sang-Ho,6.6,4,26.4
...,...,...,...
Ярополк Лапшин,10.0,1,10.0
پیمان معادی,6.0,1,6.0
塩谷 直義,7.2,1,7.2


In [487]:
bestdirek['Rank (Vote average)'] = bestdirek['vote_averages'].rank(ascending = False)
bestdirek['Rank (Sum)'] = bestdirek['sum of scores'].rank(ascending = False)
bestdirek['Average Rank'] = (bestdirek['Rank (Vote average)'] + bestdirek['Rank (Sum)'])/2
bestdirek.sort_values(by=['Average Rank'], ascending = True, inplace= True)
bestdirek['Final Ranking'] = bestdirek['Average Rank'].rank()

In [519]:
bestdirek.to_csv('Data/Director Ranking.csv')
bestdirek.to_csv('Director Ranking.csv')
bestdirek

,vote_averages,no. of entries,sum of scores,Rank (Vote average),Rank (Sum),Average Rank,Final Ranking
direk,,,,,,,
Sergiu Nicolaescu,7.800000,13,101.4,818.0,257.0,537.50,1.0
Hayao Miyazaki,7.700000,14,107.8,889.0,225.5,557.25,2.0
Rocco Urbisci,7.822222,9,70.4,720.0,506.0,613.00,3.0
Don Hertzfeldt,8.066667,6,48.4,392.0,955.0,673.50,4.0
Ertem Eğilmez,7.722222,9,69.5,834.0,518.5,676.25,5.0
...,...,...,...,...,...,...,...
Steven-Charles Jaffe,0.000000,1,0.0,17032.5,17033.0,17032.75,17032.5
Walter Klenhard,0.000000,1,0.0,17032.5,17033.0,17032.75,17032.5
Rogelio A. González,0.000000,1,0.0,17032.5,17033.0,17032.75,17032.5


In [398]:
bestdirek.shape

(17572, 2)

### Cast

In [490]:
def getCast(row, order):
    a = next((item for item in row if item['order'] == order), None)
    if a is None:
        return None
    else:
        return a['name']

In [491]:
cast = ds2[["cast", "id"]]
cast["castdict"] = cast.apply(lambda row : eval(row['cast']), axis = 1)
cast.drop('cast', inplace = True, axis =1)
cast['actor1'] = cast.castdict.apply(lambda row : getCast(row, 0))
cast['actor2'] = cast.castdict.apply(lambda row : getCast(row, 1))
cast.drop('castdict', inplace = True, axis = 1)

In [497]:
merged1 = pd.merge(cast, vote, on=['id'] )
merged1.drop('id', inplace = True, axis = 1)
a = merged1.groupby('actor1').mean()
b = merged1.groupby('actor1').count()
c = merged1.groupby('actor1').sum()
star = pd.merge(a,b, on=['actor1'])
star = pd.merge(star,c, on=['actor1'])
star.rename(columns = {'vote_average_x': 'vote_averages', 'vote_average_y': 'no. of entries','vote_average': 'sum of scores'}, inplace = True)
star.drop('actor2', inplace = True, axis = 1)
star

,vote_averages,no. of entries,sum of scores
actor1,,,
Bill Bunten,7.300,1,7.3
Lee Ji-eun,6.300,1,6.3
Ye Ji-won,7.800,1,7.8
"""Weird Al"" Yankovic",6.700,1,6.7
'Crazy' Tim Credeur,5.800,1,5.8
...,...,...,...
Юлия Волкова,1.500,1,1.5
هستی مهدوی‌فر,0.000,1,0.0
杰森·莫玛,5.500,2,11.0


In [516]:
star['Rank (Vote average)'] = star['vote_averages'].rank(ascending = False)
star['Rank (Sum)'] = star['sum of scores'].rank(ascending = False)
star['Average Rank'] = (star['Rank (Vote average)'] + star['Rank (Sum)'])/2
star.sort_values(by=['Average Rank'], ascending = True, inplace= True)
star['Final Ranking'] = star['Average Rank'].rank()
star.to_csv('Data/Star Actor.csv')
star.to_csv('Star Actor.csv')
star

,vote_averages,no. of entries,sum of scores,Rank (Vote average),Rank (Sum),Average Rank,Final Ranking
actor1,,,,,,,
Sergiu Nicolaescu,7.900000,6,47.4,463.0,773.0,618.00,1.0
Toshirō Mifune,7.439130,23,171.1,1165.0,84.0,624.50,2.0
Paul Scofield,7.833333,6,47.0,522.5,780.0,651.25,3.0
Bill Hicks,7.816667,6,46.9,525.0,781.5,653.25,4.0
Mariko Okada,7.980000,5,39.9,457.0,978.5,717.75,5.0
...,...,...,...,...,...,...,...
Germán Kraus,0.000000,1,0.0,14750.5,14751.0,14750.75,14750.5
Mary Ann Mobley,0.000000,1,0.0,14750.5,14751.0,14750.75,14750.5
Jack Beers,0.000000,1,0.0,14750.5,14751.0,14750.75,14750.5


In [517]:
merged2 = pd.merge(cast, vote, on=['id'] )
merged2.drop('id', inplace = True, axis = 1)
a = merged2.groupby('actor2').mean()
b = merged2.groupby('actor2').count()
c = merged2.groupby('actor2').sum()
costar = pd.merge(a,b, on=['actor2'])
costar = pd.merge(costar,c, on=['actor2'])
costar.rename(columns = {'vote_average_x': 'vote_averages', 'vote_average_y': 'no. of entries','vote_average': 'sum of scores'}, inplace = True)
costar.drop('actor1', inplace = True, axis = 1)
costar

,vote_averages,no. of entries,sum of scores
actor2,,,
\tRobert Osth,6.00,1,6.0
Dippanita Sharma,6.00,1,6.0
Hlubi Mboya,4.60,1,4.6
Lee Hee-joon,8.00,1,8.0
Sara Arjun,6.70,1,6.7
...,...,...,...
松田龙平,6.30,2,12.6
松隆子,6.75,2,13.5
连姆·尼森,5.80,1,5.8


In [518]:
costar['Rank (Vote average)'] = costar['vote_averages'].rank(ascending = False)
costar['Rank (Sum)'] = costar['sum of scores'].rank(ascending = False)
costar['Average Rank'] = (costar['Rank (Vote average)'] + costar['Rank (Sum)'])/2
costar.sort_values(by=['Average Rank'], ascending = True, inplace= True)
costar['Final Ranking'] = costar['Average Rank'].rank()
costar.to_csv('Data/Co-Star Actor.csv')
costar.to_csv('Co-Star Actor.csv')
costar

,vote_averages,no. of entries,sum of scores,Rank (Vote average),Rank (Sum),Average Rank,Final Ranking
actor2,,,,,,,
Leon Russell,9.666667,3,29.0,67.0,1462.5,764.75,1.0
Stan Brakhage,8.733333,3,26.2,175.0,1693.0,934.00,2.0
Irene Worth,8.000000,4,32.0,653.5,1264.0,958.75,3.0
Sofie Gråbøl,8.500000,3,25.5,241.0,1767.0,1004.00,4.0
Werner Herzog,7.409091,11,81.5,1861.0,151.0,1006.00,5.0
...,...,...,...,...,...,...,...
Ellen ten Damme,0.000000,1,0.0,21452.0,21453.0,21452.50,21452.0
Imogen Annesley,0.000000,1,0.0,21452.0,21453.0,21452.50,21452.0
Irma Huntus,0.000000,1,0.0,21452.0,21453.0,21452.50,21452.0
